In [1]:
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from utils import bracket_0, rfc_W
from common import Bracket
from seeding import Seeding

In [2]:
bracket = Bracket.NaiveBracket(bracket_0(), rfc_W())
bracket.score()

1.7312309603721477e-15

In [3]:
W = rfc_W()
b = bracket_0()
_i, _j = np.meshgrid(range(64), range(64))
mat = np.zeros((64, 64))
for (ni, i) in tqdm(list(enumerate(_i))):
    for nj, j in enumerate(_j):
        mat[ni, nj] = W[b[_i[ni, nj]], b[_j[ni, nj]]]
df = pd.DataFrame(mat, columns=[i.name for i in b], index=[i.name + "_l" for i in b])
df.style.background_gradient(cmap='Blues')

100%|██████████| 64/64 [01:03<00:00,  1.01it/s]


,Alabama,Texas A&M-Corpus Christi,Maryland,West Virginia,San Diego State,College of Charleston,Virginia,Furman,Creighton,NC State,Baylor,UC Santa Barbara,Missouri,Utah State,Arizona,Princeton,Purdue,Fairleigh Dickinson,Memphis,Florida Atlantic,Duke,Oral Roberts,Tennessee,Louisiana,Kentucky,Providence,Kansas State,Montana State,Michigan State,Southern California,Marquette,Vermont,Houston,Northern Kentucky,Iowa,Auburn,Miami (FL),Drake,Indiana,Kent State,Iowa State,Pittsburgh,Xavier,Kennesaw State,Texas A&M,Penn State,Texas,Colgate,Kansas,Howard,Arkansas,Illinois,Saint Mary's (CA),Virginia Commonwealth,Connecticut,Iona,TCU,Arizona State,Gonzaga,Grand Canyon,Northwestern,Boise State,UCLA,UNC Asheville
Alabama_l,0.560000,0.360000,0.460000,0.420000,0.540000,0.450000,0.500000,0.460000,0.460000,0.470000,0.460000,0.330000,0.460000,0.460000,0.490000,0.380000,0.510000,0.280000,0.520000,0.430000,0.500000,0.470000,0.440000,0.400000,0.450000,0.480000,0.430000,0.450000,0.440000,0.470000,0.480000,0.380000,0.520000,0.380000,0.420000,0.470000,0.470000,0.420000,0.510000,0.480000,0.440000,0.470000,0.410000,0.400000,0.420000,0.390000,0.500000,0.390000,0.650000,0.340000,0.470000,0.490000,0.530000,0.480000,0.580000,0.460000,0.460000,0.470000,0.520000,0.400000,0.510000,0.470000,0.590000,0.470000
Texas A&M-Corpus Christi_l,0.740000,0.510000,0.620000,0.570000,0.680000,0.670000,0.610000,0.660000,0.660000,0.620000,0.630000,0.590000,0.550000,0.650000,0.740000,0.590000,0.730000,0.400000,0.630000,0.690000,0.640000,0.730000,0.600000,0.610000,0.630000,0.640000,0.530000,0.700000,0.590000,0.640000,0.580000,0.590000,0.720000,0.490000,0.610000,0.600000,0.590000,0.620000,0.660000,0.560000,0.510000,0.600000,0.620000,0.640000,0.560000,0.570000,0.600000,0.610000,0.660000,0.520000,0.570000,0.630000,0.660000,0.540000,0.680000,0.600000,0.560000,0.620000,0.700000,0.560000,0.610000,0.610000,0.680000,0.680000
Maryland_l,0.630000,0.370000,0.480000,0.460000,0.640000,0.410000,0.490000,0.460000,0.540000,0.430000,0.500000,0.410000,0.480000,0.530000,0.570000,0.430000,0.580000,0.320000,0.500000,0.510000,0.460000,0.550000,0.460000,0.450000,0.510000,0.460000,0.450000,0.620000,0.440000,0.460000,0.510000,0.480000,0.600000,0.410000,0.430000,0.520000,0.460000,0.490000,0.520000,0.480000,0.480000,0.450000,0.500000,0.490000,0.400000,0.370000,0.540000,0.490000,0.600000,0.420000,0.520000,0.530000,0.590000,0.410000,0.500000,0.530000,0.490000,0.480000,0.620000,0.390000,0.480000,0.530000,0.570000,0.450000
West Virginia_l,0.610000,0.430000,0.580000,0.520000,0.590000,0.430000,0.510000,0.510000,0.570000,0.490000,0.510000,0.470000,0.440000,0.570000,0.630000,0.510000,0.600000,0.290000,0.480000,0.550000,0.590000,0.500000,0.500000,0.500000,0.600000,0.600000,0.400000,0.600000,0.500000,0.510000,0.500000,0.500000,0.630000,0.420000,0.480000,0.480000,0.460000,0.500000,0.570000,0.490000,0.420000,0.530000,0.520000,0.510000,0.450000,0.460000,0.490000,0.540000,0.590000,0.420000,0.460000,0.540000,0.560000,0.480000,0.550000,0.450000,0.470000,0.510000,0.610000,0.410000,0.510000,0.590000,0.570000,0.470000
San Diego State_l,0.560000,0.380000,0.430000,0.400000,0.500000,0.450000,0.480000,0.480000,0.420000,0.430000,0.440000,0.380000,0.460000,0.450000,0.450000,0.410000,0.480000,0.300000,0.470000,0.480000,0.480000,0.560000,0.420000,0.470000,0.420000,0.450000,0.430000,0.500000,0.420000,0.410000,0.460000,0.420000,0.560000,0.390000,0.390000,0.420000,0.430000,0.470000,0.500000,0.520000,0.420000,0.400000,0.430000,0.440000,0.400000,0.370000,0.470000,0.410000,0.600000,0.370000,0.460000,0.440000,0.510000,0.440000,0.520000,0.460000,0.440000,0.420000,0.510000,0.390000,0.460000,0.450000,0.570000,0.490000
College of Charleston_l,0.630000,0.350000,0.380000,0.430000,0.520000,0.520000,0.490000,0.390000,0.450000,0.450000,0.530000,0.330000,0.410000,0.390000,0.490000,0.340000,0.540000,0.300000,0.480000,0.580000,0.440000,0.560000,0.510000,0.400000,0.440000,0.410000,0.520000,0.330000,0.420000,0.500000,0.420000,0.350000,0.540000,0.280000,0

In [4]:
d = df.T.mean().sort_values()
t = [list(filter(lambda x: x.name == i.split("_")[0], b))[0] for i in d.index.tolist()]
s = Seeding(t, W)
print(s)
print(s.mean_variance(iters=20000))
print(s.find_maximimum_likelihood_bracket(iters=3000, verbose=True).score())
print(s.mlb.depth_error())
print(s.mlb.outcome_error())

(1) Houston
(1) Saint Mary's (CA)
(1) UCLA
(1) Marquette
(2) Purdue
(2) Connecticut
(2) College of Charleston
(2) Kansas State
(3) Texas
(3) Baylor
(3) San Diego State
(3) Tennessee
(4) Gonzaga
(4) Alabama
(4) Kansas
(4) Indiana
(5) Virginia
(5) Oral Roberts
(5) Maryland
(5) Boise State
(6) Florida Atlantic
(6) Kentucky
(6) Arizona
(6) Creighton
(7) Virginia Commonwealth
(7) West Virginia
(7) Texas A&M
(7) Arkansas
(8) Utah State
(8) Northwestern
(8) Miami (FL)
(8) UC Santa Barbara
(9) Duke
(9) Iowa State
(9) Xavier
(9) NC State
(10) Providence
(10) Southern California
(10) Michigan State
(10) Memphis
(11) Furman
(11) Kent State
(11) Illinois
(11) Arizona State
(12) Iowa
(12) TCU
(12) Louisiana
(12) Drake
(13) Penn State
(13) Vermont
(13) Iona
(13) Auburn
(14) Pittsburgh
(14) UNC Asheville
(14) Northern Kentucky
(14) Missouri
(15) Texas A&M-Corpus Christi
(15) Grand Canyon
(15) Howard
(15) Colgate
(16) Princeton
(16) Montana State
(16) Kennesaw State
(16) Fairleigh Dickinson


score: 3.3900763133482688e-18: 100%|██████████| 20000/20000 [00:10<00:00, 1820.29it/s]


-277.52100292115983


score: 5.618174080658429e-15: 100%|██████████| 3000/3000 [00:01<00:00, 1900.08it/s]


5.618174080658429e-15
139
225


In [5]:
ncaa_seed = Seeding(Seeding.inverse_arrange(bracket_0()), W)
print(ncaa_seed)
print(ncaa_seed.mean_variance(iters=20000))
print(ncaa_seed.find_maximimum_likelihood_bracket(iters=3000, verbose=True).score())
print(ncaa_seed.mlb.depth_error())
print(ncaa_seed.mlb.outcome_error())

(1) Alabama
(1) Houston
(1) Kansas
(1) Purdue
(2) UCLA
(2) Texas
(2) Arizona
(2) Marquette
(3) Baylor
(3) Gonzaga
(3) Kansas State
(3) Xavier
(4) Connecticut
(4) Tennessee
(4) Indiana
(4) Virginia
(5) San Diego State
(5) Duke
(5) Saint Mary's (CA)
(5) Miami (FL)
(6) Iowa State
(6) Creighton
(6) Kentucky
(6) TCU
(7) Texas A&M
(7) Michigan State
(7) Missouri
(7) Northwestern
(8) Memphis
(8) Arkansas
(8) Maryland
(8) Iowa
(9) Florida Atlantic
(9) West Virginia
(9) Auburn
(9) Illinois
(10) Boise State
(10) Penn State
(10) Southern California
(10) Utah State
(11) NC State
(11) Providence
(11) Pittsburgh
(11) Arizona State
(12) College of Charleston
(12) Oral Roberts
(12) Drake
(12) Virginia Commonwealth
(13) Kent State
(13) Iona
(13) Furman
(13) Louisiana
(14) Kennesaw State
(14) UC Santa Barbara
(14) Grand Canyon
(14) Montana State
(15) Vermont
(15) Colgate
(15) Princeton
(15) UNC Asheville
(16) Fairleigh Dickinson
(16) Howard
(16) Northern Kentucky
(16) Texas A&M-Corpus Christi


score: 1.3752596622964877e-17: 100%|██████████| 20000/20000 [00:10<00:00, 1850.55it/s]


-276.02988617988746


score: 9.99733909710334e-15: 100%|██████████| 3000/3000 [00:01<00:00, 1928.40it/s] 


9.99733909710334e-15
117
243


In [6]:
with open("./seeding_optim_no_temp.pkl", "rb") as doc:
    optimal_seed = pickle.load(doc)[-1]

In [7]:
print(optimal_seed)
print(optimal_seed.mean_variance(iters=20000))
print(optimal_seed.find_maximimum_likelihood_bracket(iters=3000, verbose=True).score())
print(optimal_seed.mlb.depth_error())
print(optimal_seed.mlb.outcome_error())

(1) Purdue
(1) Houston
(1) Kansas
(1) Alabama
(2) UCLA
(2) Texas
(2) Arizona
(2) Marquette
(3) Baylor
(3) Gonzaga
(3) Kansas State
(3) San Diego State
(4) Connecticut
(4) Tennessee
(4) Indiana
(4) Virginia
(5) Xavier
(5) Duke
(5) Saint Mary's (CA)
(5) Miami (FL)
(6) Iowa State
(6) Creighton
(6) Kentucky
(6) TCU
(7) Texas A&M
(7) Michigan State
(7) Missouri
(7) Northwestern
(8) Memphis
(8) Arkansas
(8) Maryland
(8) Iowa
(9) Florida Atlantic
(9) West Virginia
(9) Auburn
(9) Illinois
(10) Penn State
(10) Boise State
(10) Southern California
(10) Utah State
(11) NC State
(11) Providence
(11) Mississippi State
(11) College of Charleston
(12) Nevada
(12) Oral Roberts
(12) Virginia Commonwealth
(12) Drake
(13) Kent State
(13) Iona
(13) Furman
(13) Grand Canyon
(14) Kennesaw State
(14) UC Santa Barbara
(14) Louisiana
(14) Colgate
(15) Vermont
(15) Montana State
(15) Princeton
(15) UNC Asheville
(16) Texas Southern
(16) Howard
(16) Northern Kentucky
(16) Texas A&M-Corpus Christi


score: 2.4135507459552582e-17: 100%|██████████| 20000/20000 [00:11<00:00, 1793.08it/s]


-279.27993406150534


score: 5.642668154613997e-15: 100%|██████████| 3000/3000 [00:01<00:00, 1903.69it/s]


5.642668154613997e-15
117
241


In [8]:
#calculate actual tournament depth error of each seeding